In [1]:
import os
if "py" not in os.listdir("."):
    os.chdir("..")
    
%config Completer.use_jedi = False

In [9]:
import microdf as mdf
import numpy as np
import openfisca_uk as o
import pandas as pd
from openfisca_uk import IndividualSim, PopulationSim
from openfisca_uk.reforms.modelling import reported_benefits
from py.calc_ubi import ubi_reform

REGION_CODES = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]

REGION_NAMES = [
    "North East",
    "North West",
    "Yorkshire and the Humber",
    "East Midlands",
    "West Midlands",
    "East of England",
    "London",
    "South East",
    "South West",
    "Wales",
    "Scotland",
    "Northern Ireland",
]

region_code_map = dict(zip(range(len(REGION_CODES)), REGION_CODES))
region_name_map = dict(zip(range(len(REGION_NAMES)), REGION_NAMES))

optimal_params = pd.read_csv("optimal_params.csv")  # Up a folder.


def reform(i):
    row = optimal_params.iloc[i].round()
    return ubi_reform(
        adult=row.adult,
        child=row.child,
        senior=row.senior,
        dis_base=row.dis_base,
        geo=row[REGION_CODES],
    )


reforms = [reform(i) for i in range(3)]

baseline_sim = PopulationSim(reported_benefits)
reform_sims = [PopulationSim(reported_benefits, reform) for reform in reforms]

REFORM_NAMES = ["1: Foundational", "2: Disability", "3: Disability + geo"]

BASELINE_PERSON_COLS = [
    "household_weight",
    "age",
    "region",
    "is_disabled_for_ubi",
]

# Extract these for baseline too.
REFORM_PERSON_COLS = [
    "household_net_income",
    "in_poverty_bhc",
    "in_deep_poverty_bhc",
]

BASELINE_HH_COLS = ["household_weight", "people_in_household", "region"]

# Extract these for baseline too.
REFORM_HH_COLS = [
    "household_net_income",
    "equiv_household_net_income",
    "poverty_gap_bhc",
    "poverty_gap_ahc",
]

p_base = mdf.MicroDataFrame(
    baseline_sim.df(
        BASELINE_PERSON_COLS + REFORM_PERSON_COLS, map_to="person"
    ),
    weights="household_weight",
)
p_base.rename(
    dict(zip(REFORM_PERSON_COLS, [i + "_base" for i in REFORM_PERSON_COLS])),
    axis=1,
    inplace=True,
)

hh_base = mdf.MicroDataFrame(
    baseline_sim.df(BASELINE_HH_COLS + REFORM_HH_COLS, map_to="household"),
    weights="household_weight",
)
hh_base.rename(
    dict(zip(REFORM_HH_COLS, [i + "_base" for i in REFORM_HH_COLS])),
    axis=1,
    inplace=True,
)
hh_base["person_weight"] = (
    hh_base.household_weight * hh_base.people_in_household
)

# # Add region code and names
# hh_base["region_code"] = hh_base.region.map(region_code_map)
# hh_base["region_name"] = hh_base.region.map(region_name_map)
# p_base["region_code"] = p_base.region.map(region_code_map)
# p_base["region_name"] = p_base.region.map(region_name_map)

# Change weight column to represent people for decile groups.
hh_base.set_weights(hh_base.person_weight)
hh_base["decile"] = np.ceil(
    hh_base.equiv_household_net_income_base.rank(pct=True) * 10
)

# Change weight back to household weight for correct calculation of totals.
hh_base.set_weights(hh_base.household_weight)


def reform_p(i):
    p = reform_sims[i].df(REFORM_PERSON_COLS, map_to="person")
    p["reform"] = REFORM_NAMES[i]
    return mdf.concat([p_base, p], axis=1)


def reform_hh(i):
    hh = reform_sims[i].df(REFORM_HH_COLS, map_to="household")
    hh["reform"] = REFORM_NAMES[i]
    return mdf.concat([hh_base, hh], axis=1)


def pct_chg(base, new):
    return (new - base) / base


def reform_stats(df):
    # For applying over a groupby(reform) or a .
    gini = df.equiv_household_net_income_base.aggregate(
        ["gini", "top_10pct_share"]
    )
    p_agg = df[["household_net_income_pl", "winner"]].mean()



p = mdf.concat([reform_p(i) for i in range(3)])
h = mdf.concat([reform_hh(i) for i in range(3)])

p = p.reset_index(drop=True)#.drop("index", axis=1)
h = h.reset_index(drop=True)#.drop("index", axis=1)

def chg(df, col):
    df[col + "_chg"] = df[col] - df[col + "_base"]
    # Percentage change, only defined for positive baselines.
    df[col + "_pc"] = np.where(
        df[col + "_base"] > 0,
        df[col + "_chg"] / df[col + "_base"],
        np.nan,
    )
    # Percentage loss. NB: np.minimum(np.nan, 0) -> np.nan.
    df[col + "_pl"] = np.minimum(0, df[col + "_pc"])

chg(p, "household_net_income")
chg(h, "household_net_income")
p["winner"] = p.household_net_income_chg > 0
h["winner"] = h.household_net_income_chg > 0
# Per-reform.
INEQS = ["gini", "top_10_pct_share", "top_1_pct_share"]
ineq_base = h.groupby("reform").equiv_household_net_income_base.agg(INEQS)
ineq_base.columns = [i + "_base" for i in ineq_base.columns]
ineq_reform = h.groupby("reform").equiv_household_net_income.agg(INEQS)
ineq_reform.columns = [i + "_reform" for i in ineq_reform.columns]
p_agg = p.groupby("reform")[["household_net_income_pl", "winner"]].mean()
r = p_agg.join(ineq_base).join(ineq_reform, on="reform")
r["reform"] = r.index  # Easier for plotting.
for i in INEQS:
    r[i + "_pc"] = pct_chg(r[i + "_base"], r[i + "_reform"])

# Per reform per decile (by household).

decile = (
    h.groupby(["reform", "decile"])
    .sum()[
        [
            "household_net_income",
            "household_net_income_base",
            "people_in_household",
        ]
    ]
    .reset_index()
)
decile["chg"] = (
    decile.household_net_income - decile.household_net_income_base
)
decile["chg_pp"] = decile.chg / decile.people_in_household
decile["pc"] = decile.chg / decile.household_net_income_base
# return p, h, r, decile

In [3]:
h = mdf.concat([reform_hh(i) for i in range(3)])

In [10]:
h

,household_weight,people_in_household,region,household_net_income_base,equiv_household_net_income_base,poverty_gap_bhc_base,poverty_gap_ahc_base,person_weight,decile,household_net_income,equiv_household_net_income,poverty_gap_bhc,poverty_gap_ahc,reform,household_net_income_chg,household_net_income_pc,household_net_income_pl,winner,__tmp_weights
0,1329.0,1,7,53660.421875,80090.179688,0.00000,0.000000,1329.0,10.0,41920.789062,62568.339844,0.000000,0.000000,1: Foundational,-11739.632812,-0.218776,-0.218776,False,1329.0
1,1137.0,3,1,19646.113281,16371.760742,0.00000,1081.087646,3411.0,2.0,16921.771484,14101.475586,1486.228149,3805.429932,1: Foundational,-2724.341797,-0.138671,-0.138671,False,1137.0
2,1183.0,1,2,8038.000000,11997.014648,2239.80127,3648.480225,1183.0,1.0,8240.797852,12299.698242,2037.001465,3445.682617,1: Foundational,202.797852,0.025230,0.000000,True,1183.0
3,1301.0,1,2,14399.839844,21492.296875,0.00000,0.000000,1301.0,4.0,11564.796875,17260.890625,0.000000,1421.682861,1: Foundational,-2835.042969,-0.196880,-0.196880,False,1301.0
4,1640.0,1,2,53834.640625,80350.210938,0.00000,0.000000,1640.0,10.0,40064.000000,59797.011719,0.000000,0.000000,1: Foundational,-13770.640625,-0.255795,-0.255795,False,1640.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57502,1241.0,1,9,19210.000000,28671.640625,0.00000,0.000000,1241.0,5.0,9236.673828,13786.080078,1041.126709,6349.807617,3: Disability + geo,-9973.326172,-0.519174,-0.519174,False,1241.0
57503,2282.0,4,6,81767.500000,58405.359375,0.00000,0.000000,9128.0,9.0,78467.656250,56048.328125,0.000000,0.000000,3: Disability + geo,-3299.843750,-0.040356,-0.040356,False,2282.0
57504,524.0,4,11,27460.050781,22883.375000,0.00000,0.000000,2096.0,4.0,35781.203125,29817.667969,0.000000,0.000000,3: Disability + geo,8321.152344,0.303028,0.000000,True,524.0
57505,1379.0,1,7,33710.730469,50314.523438,0.00000,0.000000,1379.0,8.0,24436.000000,36471.640625,0.000000,0.000000,3: Disability + geo,-9274.730469,-0.275127,-0.275127,False,1379.0


In [5]:
h.reset_index().drop("index", axis=1).__class__

microdf.generic.MicroDataFrame

In [6]:
h.groupby("reform").equiv_household_net_income_base.mean()

reform
1: Foundational        34792.426049
2: Disability          34792.426049
3: Disability + geo    34792.426049
dtype: float64

In [7]:
h[["people_in_household"]].sum()

people_in_household    196435665.0
dtype: float64

In [8]:
h.groupby("decile").sum()

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
h.reform.rank()

In [ ]:
d = mdf.MicroDataFrame({"x": [1, 2, 3]}, index=[1, 1, 2], weights=pd.Series([4, 5, 6], index=[1, 1, 2]))

In [ ]:
d.gini()

In [ ]:
h = h.reset_index().drop("index", axis=1)

In [ ]:
h.index

In [ ]:
h[h.household_net_income_base > 0][["reform", "decile", "household_net_income_base", "household_net_income", "household_weight"]].groupby("reform").sum()

In [ ]:
h[h.household_net_income_base > 0][["reform", "decile", "household_net_income_base", "household_net_income", "household_weight"]].groupby(["reform", "decile"]).sum()

In [ ]:
decile = (
        # Remove non-positive income, for whom income change is unhelpful.
        h[h.household_net_income_base > 0]
        .groupby(["reform", "decile"])[
            [
                "household_net_income_base",
                "household_net_income",
                "household_weight",
            ]
        ]
        .sum()
        .reset_index()
    )
decile["chg"] = (
    decile.household_net_income - decile.household_net_income_base
)
decile["chg_per_hh"] = decile.chg / decile.household_weight
decile["pc"] = decile.chg / decile.household_net_income_base

In [ ]:
pd.DataFrame(hh_base).groupby("decile").person_weight.sum()

In [ ]:
hh_base.equiv_household_net_income_base < (295 * 52)

In [ ]:
((hh_base.equiv_household_net_income_base < (295 * 52)) != hh_base.in_poverty_bhc).mean()

In [ ]:
(hh_base.equiv_household_net_income_base < hh_base.poverty_line_bhc) != hh_base.in_poverty_bhc

In [ ]:
o.tools.aggregates.poverty_rate(baseline_sim)